In [21]:
pip install plotly_express

Note: you may need to restart the kernel to use updated packages.


In [30]:
import pandas as pd 
import geopandas as gpd
import numpy as np
from folium.features import GeoJsonTooltip
from folium.features import GeoJsonPopup
import pandas as pd
import geopy
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import matplotlib.pyplot as plt
import plotly_express as px
import tqdm
from tqdm._tqdm_notebook import tqdm_notebook

On  télécharge un fichier geojson issu du site du gouvernement représentant les arrondissements de Paris. On le transforme dans un premier temps en un DataFrame grâce à geopandas. 

In [31]:
geojson = gpd.read_file('arrondissements.geojson')


In [32]:
geojson.head()

,c_ar,l_aroff,surface,l_ar,n_sq_co,c_arinsee,n_sq_ar,perimetre,geometry
0,11,Popincourt,3.665442e+06,11ème Ardt,750001537,75111,750000011,8282.011886,"POLYGON ((2.39624 48.85415, 2.39708 48.85308, ..."
1,2,Bourse,9.911537e+05,2ème Ardt,750001537,75102,750000002,4554.104360,"POLYGON ((2.35152 48.86443, 2.35095 48.86341, ..."
2,6,Luxembourg,2.153096e+06,6ème Ardt,750001537,75106,750000006,6483.686786,"POLYGON ((2.34459 48.85405, 2.34428 48.85375, ..."
3,12,Reuilly,1.631478e+07,12ème Ardt,750001537,75112,750000012,24089.666298,"POLYGON ((2.41388 48.83357, 2.41401 48.83357, ..."
4,17,Batignolles-Monceau,5.668835e+06,17ème Ardt,750001537,75117,750000017,10775.579516,"POLYGON ((2.29517 48.87396, 2.29504 48.87378, ..."


On récupère nos données immobilières  

In [33]:
df = pd.read_csv("Base_WS_traitee.csv")
print(df.shape)

(9012, 8)


In [34]:
df.head()

,Unnamed: 0,date_mutation,valeur_fonciere,type_local,surface_reelle_bati,nombre_pieces_principales,longitude,latitude
0,0,29/06/2022,740000,Appartement,76,3,2.257874,48.839415
1,1,28/06/2022,598000,Appartement,54,2,2.260871,48.838515
2,2,28/06/2022,620874,Appartement,85,3,2.257995,48.837945
3,3,23/06/2022,230000,Appartement,30,2,2.260568,48.838470
4,4,17/06/2022,1380000,Appartement,141,5,2.256915,48.839126


On cherche à connaitre l'arrondissement des biens immobiliers à partir de leurs coordonnées. Pour cela, on utilise un géocodage inverse.

In [35]:
locator = Nominatim(user_agent="myGeocoder")
coordinates = "48.839415, 2.257874"
location = locator.reverse(coordinates)
location.raw


{'place_id': 9674377,
 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright',
 'osm_type': 'node',
 'osm_id': 1030121554,
 'lat': '48.8394147',
 'lon': '2.2578735',
 'display_name': "127, Rue Michel-Ange, Quartier d'Auteuil, Paris 16e Arrondissement, Paris, Île-de-France, France métropolitaine, 75016, France",
 'address': {'house_number': '127',
  'road': 'Rue Michel-Ange',
  'suburb': "Quartier d'Auteuil",
  'city_district': 'Paris',
  'city': 'Paris',
  'ISO3166-2-lvl6': 'FR-75',
  'state': 'Île-de-France',
  'ISO3166-2-lvl4': 'FR-IDF',
  'region': 'France métropolitaine',
  'postcode': '75016',
  'country': 'France',
  'country_code': 'fr'},
 'boundingbox': ['48.8393647', '48.8394647', '2.2578235', '2.2579235']}

In [36]:
print(location.address[len(location.address)-13:len(location.address)-8])

75016


In [39]:
Arrondissement = []
for i in range(9012):
    la = df.latitude[i]
    lo = df.longitude[i]
    li=[la,lo]
    locator = Nominatim(user_agent="myGeocoder")
    coordinates = li
    location = locator.reverse(coordinates)
    location.raw
    Arrondissement.append(location.address[len(location.address)-13:len(location.address)-8])
    
print(Arrondissement)

GeocoderUnavailable: HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /reverse?lat=48.866431&lon=2.348095&format=json&addressdetails=1 (Caused by ReadTimeoutError("HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Read timed out. (read timeout=1)"))

In [226]:
df1 = pd.DataFrame((zip(Arrondissement,df["Unnamed: 0"])), columns = ['Arrondissement','index'])

In [227]:
df1.head()

,Arrondissement,index
0,75016,0
1,75016,1
2,75016,2
3,75016,3
4,75016,4


In [229]:
df2 = df.merge(df1, left_on="Unnamed: 0", right_on="index", how="outer")

In [230]:
df2.head()

,Unnamed: 0,date_mutation,valeur_fonciere,type_local,surface_reelle_bati,nombre_pieces_principales,longitude,latitude,Arrondissement,index
0,0,29/06/2022,740000,Appartement,76,3,2.257874,48.839415,75016,0
1,1,28/06/2022,598000,Appartement,54,2,2.260871,48.838515,75016,1
2,2,28/06/2022,620874,Appartement,85,3,2.257995,48.837945,75016,2
3,3,23/06/2022,230000,Appartement,30,2,2.260568,48.838470,75016,3
4,4,17/06/2022,1380000,Appartement,141,5,2.256915,48.839126,75016,4


On rajoute les données propres au prix au mètre carré de chaque bien.

In [222]:
Prix_m2 = []
for i in range(5952):
    Prix_m2.append(df.valeur_fonciere[i]/df.surface_reelle_bati[i])


    

In [231]:
df3 = pd.DataFrame((zip(Prix_m2,df["Unnamed: 0"])), columns = ['Prix_m2','index1'])

In [232]:
df3.head()

,Prix_m2,index1
0,9736.842105,0
1,11074.074074,1
2,7304.400000,2
3,7666.666667,3
4,9787.234043,4


In [233]:
df4 = df2.merge(df3, left_on="index", right_on="index1", how="outer")
df4.head()

,Unnamed: 0,date_mutation,valeur_fonciere,type_local,surface_reelle_bati,nombre_pieces_principales,longitude,latitude,Arrondissement,index,Prix_m2,index1
0,0,29/06/2022,740000,Appartement,76,3,2.257874,48.839415,75016,0,9736.842105,0
1,1,28/06/2022,598000,Appartement,54,2,2.260871,48.838515,75016,1,11074.074074,1
2,2,28/06/2022,620874,Appartement,85,3,2.257995,48.837945,75016,2,7304.400000,2
3,3,23/06/2022,230000,Appartement,30,2,2.260568,48.838470,75016,3,7666.666667,3
4,4,17/06/2022,1380000,Appartement,141,5,2.256915,48.839126,75016,4,9787.234043,4


On supprime les lignes du DataFrame qui n'ont pas la donnée code postal

In [234]:
df4['Arrondissement'] = df4['Arrondissement'].fillna(0)

In [267]:
list = []
a = -1
for i in df4['Arrondissement']: 
    a = a+1
    try : 
        i = int(i)
    except ValueError : 
        list.append(a)
        
df4 = df4.drop(df4.index[list])

#On vérifie qu'il ne reste pas de code postal au format incorrect dans arrondissement 
list2 = []
a = -1
for i in df7['Arrondissement']: 
    a = a + 1 
    try : 
        list2.append(int(i))
    except ValueError : 
        print(i)
    


df4['Arrondissement'] = df4['Arrondissement'].astype('int64')

On récupère le numéro d'arrondissement à partir du code postal.

In [268]:
Arrondissement_1 = []
for i in df4['Arrondissement'] : 
    Arrondissement_1.append(i%100)
    

df4['Arrondissement'] = Arrondissement_1

In [270]:
df4.drop(df4[(df4['Arrondissement'] == 0)].index, inplace=True)

df4

,Unnamed: 0,date_mutation,valeur_fonciere,type_local,surface_reelle_bati,nombre_pieces_principales,longitude,latitude,Arrondissement,index,Prix_m2,index1
0,0,29/06/2022,740000,Appartement,76,3,2.257874,48.839415,16,0,9736.842105,0
1,1,28/06/2022,598000,Appartement,54,2,2.260871,48.838515,16,1,11074.074074,1
2,2,28/06/2022,620874,Appartement,85,3,2.257995,48.837945,16,2,7304.400000,2
3,3,23/06/2022,230000,Appartement,30,2,2.260568,48.838470,16,3,7666.666667,3
4,4,17/06/2022,1380000,Appartement,141,5,2.256915,48.839126,16,4,9787.234043,4
...,...,...,...,...,...,...,...,...,...,...,...,...
5947,5947,11/10/2021,234000,Appartement,23,1,2.408722,48.862658,20,5947,10173.913043,5947
5948,5948,25/06/2021,492500,Appartement,43,2,2.408717,48.868228,20,5948,11453.488372,5948
5949,5949,20/05/2021,563400,Appartement,65,3,2.408671,48.862907,20,5949,8667.692308,5949
5950,5950,23/02/2022,625000,Appartement,76,4,2.410197,48.872554,20,5950,8223.684211,5950


In [272]:
table = np.round(pd.pivot_table(df4, index = df4["Arrondissement"], 
                       values = ["Prix_m2"]))


df5 = table.reset_index()

On fusionne le DataFrame avec le geojson avec pour clé l'arrondissement (Arrondissement dans df5, c_ar dans geojson)

In [273]:
df_final = geojson.merge(df5, left_on="c_ar", right_on="Arrondissement", how="outer") 

On utilise plotly_express pour créer une carte interactive qui donne les prix moyen au mètre carré dans chaque arrondissement

In [1]:
import plotly.express as px

fig1 = px.choropleth_mapbox(df_final, locations = 'Arrondissement',
                            featureidkey = 'properties.c_ar',
                            geojson= geojson,
                            color=df_final['Prix_m2'],
                            color_continuous_scale="PuRd",
                            range_color=[9000,15000],
                            hover_name='Arrondissement',
                            hover_data=['Prix_m2'],
                            labels = {'Arrondissement' : 'Arrondissement ', 
                                     'Prix_m2' : 'Prix du m2 '},
                            title="Prix moyen du m² par arrondissement",
                            mapbox_style="open-street-map",
                            center= {'lat':48.866669, 'lon':2.33333},
                            zoom =10.5, 
                            opacity= 0.6)

fig1.show()

NameError: name 'df_final' is not defined